https://machinelearningmastery.com/text-generation-with-lstm-in-pytorch/

In [ ]:
#Acessando o googledrive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np

In [ ]:
file_name = "/content/drive/MyDrive/disciplinas/LMs/domCasmurro.txt"

In [ ]:
domcasmurro = open(file_name, encoding="utf-8").read().lower()


In [ ]:

# caracteres para inteiros
chars = sorted(list(set(domcasmurro)))
char_to_int = dict((c, i) for i, c in enumerate(chars))


n_chars = len(domcasmurro)
n_vocab = len(chars)
print("Total de caracteres: ", n_chars)
print("Total Vocab: ", n_vocab)

Total de caracteres:  371506
Total Vocab:  66


In [ ]:

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = domcasmurro[i:i + seq_length]
    seq_out = domcasmurro[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  371406


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim

# reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([371406, 100, 1]) torch.Size([371406])


In [ ]:
import torch.optim as optim
import torch.utils.data as data

class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
        #self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x


In [ ]:

torch.cuda.is_available()

True

In [ ]:

n_epochs = 40
batch_size = 128
model = CharModel()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)

best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))

        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))

torch.save([best_model, char_to_int], "single-char.pth")

Epoch 0: Cross-entropy: 1036151.9375
Epoch 1: Cross-entropy: 1004840.7500
Epoch 2: Cross-entropy: 979231.6250
Epoch 3: Cross-entropy: 948604.6875
Epoch 4: Cross-entropy: 923658.3125
Epoch 5: Cross-entropy: 897364.5625
Epoch 6: Cross-entropy: 875768.8750
Epoch 7: Cross-entropy: 857871.4375
Epoch 8: Cross-entropy: 840565.4375
Epoch 9: Cross-entropy: 826666.6250
Epoch 10: Cross-entropy: 813936.3750
Epoch 11: Cross-entropy: 803080.6875
Epoch 12: Cross-entropy: 791802.0625
Epoch 13: Cross-entropy: 782245.3750
Epoch 14: Cross-entropy: 775540.1250
Epoch 15: Cross-entropy: 763406.1875
Epoch 16: Cross-entropy: 755099.5625
Epoch 17: Cross-entropy: 749421.0000
Epoch 18: Cross-entropy: 743199.4375
Epoch 19: Cross-entropy: 737123.6250
Epoch 20: Cross-entropy: 733307.6875
Epoch 21: Cross-entropy: 728101.0625
Epoch 22: Cross-entropy: 721563.5625
Epoch 23: Cross-entropy: 723922.5625
Epoch 24: Cross-entropy: 714453.0000
Epoch 25: Cross-entropy: 709574.5000
Epoch 26: Cross-entropy: 708838.0000
Epoch 27:

In [ ]:
# Generation using the trained model
best_model, char_to_int = torch.load("single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)

# randomly generate a prompt
seq_length = 100
raw_text = open(file_name, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
pattern = [char_to_int[c] for c in prompt]

model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(1000):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x.to(device))
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")